# My spaCy, nltk and natasha notebook

Hi, this is a notebook where I practice all my NLP skills. I use the first Harry Potter book in Russian as a data set. 

In [94]:
import os
os.chdir("/Users/maria.onoeva/Desktop/new_folder/GitHub/nlp-repo")

In [207]:
path = 'questions/txts/'
all_HP = '/Users/maria.onoeva/Desktop/HP_all_ru_Spivak.txt'

with open(all_HP, encoding='utf8') as file:
    text_ru = file.read()

# cleaning text, not necessary for all

#text_ru_cleaned = text_ru_cleaned.replace('\n\n', '') #  char

import re
text_ru_cleaned = re.sub(r'\.(?!\s)', '. ', text_ru)

# Replace CRLF and other common line breaks with a space
text_ru_cleaned = text_ru_cleaned.replace('\r\n', ' ').replace('\n', ' ').replace('\r', ' ')

# Replace actual non-breaking space (Unicode \u00A0), not the string 'NBSP'
text_ru_cleaned = text_ru_cleaned.replace('\u00A0', ' ')

text_ru_cleaned = re.sub(r'\s{2,}', ' ', text_ru_cleaned)
text_ru_cleaned = text_ru_cleaned.strip()

I import Russian from spaCy and initialize it in `nlp`. It then creates a so-called `doc` object. Docs contain tokens and if I call `doc[34]`, it will return the 34th token in this doc. There is also a `span` object as below.

In [251]:
import spacy

questions_spacy = []
nlp = spacy.load("ru_core_news_sm", disable=["morphologizer", "tok2vec", "tagger", "attribute_ruler", "lemmatizer", "ner", "custom", "textcat"])
print(nlp.pipe_names)

nlp.max_length = 56065230
doc = nlp(text_ru_cleaned)

['parser']


In [255]:
questions_spacy = [sent for sent in doc.sents if '?' in sent.text]
print(questions_spacy)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Now I want to print out some sentences. Since I investigate questions, I want to print out some of them. The first step is to extract all questions or sentences with a question mark at the end. Update: now I check whether sent contains '?' because I can have '??' (well this is ok with the previous method) or '?!'.

I am not completely happy about automatic sentencing by `spaCy`. I'll try `nltk`.

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

# This is necessary because I had the error with loading nltk parts 
nltk.download('punkt_tab')
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [228]:
sentences_nltk = sent_tokenize(text_ru_cleaned, language='russian')
question_pattern = '?'
questions_nltk = [sent for sent in sentences_nltk if question_pattern in sent]

Not the best but seems better. I am going to try another tool, [Natasha](https://natasha.github.io/).

In [210]:
from natasha import Segmenter, Doc

segmenter = Segmenter()
natasha_doc = Doc(text_ru_cleaned)

natasha_sentences = natasha_doc.segment(segmenter)
questions_natasha = [sent.text for sent in natasha_doc.sents if question_pattern in sent.text]

11252


And for fun, let's do `regex`. 

In [227]:
print(f'spaCy: {len(questions_spacy)}' )
print(f'NLTK: {len(questions_nltk)}')
print(f'Natasha: {len(questions_natasha)}')

spaCy: 11324
NLTK: 11263
Natasha: 11252
regex: 11324


I want to compare questions from all three tools. 

In [238]:
from itertools import zip_longest
import csv

questions_3_tools = 'questions/csvs/questions_3_tools.csv'

with open(questions_3_tools, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    
    # Write header row
    writer.writerow(['ID', 'spaCy', 'NLTK', 'Natasha'])
    
    # Use enumerate to add an ID for each row, starting at 1
    for id_num, (spacy, nltk, natasha) in enumerate(zip_longest(questions_spacy, questions_nltk, questions_natasha, fillvalue=''), start=1):
        writer.writerow([id_num, spacy, nltk, natasha])

Then I would like to do some syntactic parsing of some questions. 